In [ ]:
import requests
import time
import re

Subtitles can be periodically fetched from the below url

In [ ]:
SUBTITLE_URL = "https://livestreaming.b67.tweedekamer.nl/live/plenairezaal/subtitles/nl_Live.m3u8?sourcetimestamps=1"

In [ ]:
r = requests.get(SUBTITLE_URL)

In [ ]:
urls = r.content
urls

In [ ]:
REGEX_UUID4 = r"[a-f0-9]{8}-?[a-f0-9]{4}-?4[a-f0-9]{3}-?[89ab][a-f0-9]{3}-?[a-f0-9]{12}"
REGEX_SUBTITLE_URL = f'https://amcpwetkms-euwe.streaming.media.azure.net\/{REGEX_UUID4}\/subtitles\/nl\/[0-9]+.vtt'

In [ ]:
urls = re.findall(REGEX_SUBTITLE_URL, str(urls))
urls

These URLs are all links to [WebVTT](https://developer.mozilla.org/en-US/docs/Web/API/WebVTT_API) files, as shown below

In [ ]:
requests.get(urls[-2]).content

Here we don't transform this text, we'll leave that to the transform phase

## Streamer

In [ ]:
import requests
import time
import re

SUBTITLE_URL = "https://livestreaming.b67.tweedekamer.nl/live/plenairezaal/subtitles/nl_Live.m3u8?sourcetimestamps=1"

REGEX_UUID4 = r"[a-f0-9]{8}-?[a-f0-9]{4}-?4[a-f0-9]{3}-?[89ab][a-f0-9]{3}-?[a-f0-9]{12}"
REGEX_SUBTITLE_URL = f'https://amcpwetkms-euwe.streaming.media.azure.net\/{REGEX_UUID4}\/subtitles\/nl\/[0-9]+.vtt'

IDLE_TIME = 2
MAX_URL_LENGTH = 200

class SubtitleStreamer:
    def __init__(self):
        self.urls_seen = []
    
    def run(self):
        while True:
            try:
                urls = self._get_new_urls()
                urls = self._update_urls_seen(urls)
                subs = self._get_new_subs(urls)
                time.sleep(IDLE_TIME)
            except KeyboardInterrupt:
                break
    
    def _get_new_urls(self):
        r = requests.get(SUBTITLE_URL)
        urls = re.findall(REGEX_SUBTITLE_URL, str(r.content))
        return urls
    
    def _get_new_subs(self, urls):
        for url in urls:
            print(requests.get(url).content)
    
    def _update_urls_seen(self, urls):
        new_urls = []
        for url in urls:
            if url not in self.urls_seen:
                self.urls_seen.append(url)
                new_urls.append(url)
        
        if len(self.urls_seen) > MAX_URL_LENGTH:
            self.urls = self.urls_seen[-MAX_URL_LENGTH:]
        
        return new_urls

In [ ]:
streamer = SubtitleStreamer()
streamer.run()